# Part 1: web scraping using python

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
tableBody = soup.find_all('tbody', limit=1)[0]

dfPostalToronto = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood'])

#### Parsing rows of html table and adding items to the data frame

In [5]:
for tag in tableBody.find_all('tr'):
    postcod = tag.find_next()
    borough = postcod.find_next()
    neighborhood = borough.find_next()
    
    str_borough = borough.string     
    
    if borough.a is not None :
      str_borough =  borough.a.string.strip()
      neighborhood = neighborhood.find_next()
        
    str_neighborhood = neighborhood.string   
    
    if neighborhood.a is not None :
      str_neighborhood =  neighborhood.a.string.strip()
    
    dfPostalToronto = dfPostalToronto.append({
                        'Postcode'     : postcod.string.strip(), 
                        'Borough'      : str_borough, 
                        'Neighborhood': str_neighborhood.strip()}, ignore_index=True)

#### Preparing data and excluding boroughs that are not assigned

In [6]:
dfPostalToronto = dfPostalToronto.iloc[1:]

dfPostalToronto = dfPostalToronto.loc[dfPostalToronto['Borough']  != 'Not assigned']

nbAux = np.where(dfPostalToronto.Neighborhood.eq('Not assigned'), dfPostalToronto.Borough, dfPostalToronto.Neighborhood)
dfPostalToronto.Neighborhood = nbAux

In [7]:
dfPostalToronto = dfPostalToronto.groupby(['Postcode', 'Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)])
dfPostalToronto = dfPostalToronto.reset_index()

In [8]:
dfPostalToronto.shape

(180, 3)

# Part 2: Finding latitude and the longitude coordinates of each neighborhood

In [10]:
GeoToronto = pd.read_csv('https://cocl.us/Geospatial_data')

In [12]:
dfPostalToronto = dfPostalToronto.merge(GeoToronto)
dfPostalToronto.drop('Postal Code', axis=1, inplace=True)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False